In [175]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [176]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option('display.max_columns', None)

In [177]:
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")

In [178]:
union_bienes = pd.concat([df_mobiliario, df_equipos_medicos, df_industriales])
union_bienes["conteo"] = 1

In [179]:
pd.pivot_table(
    union_bienes,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value="0",
    aggfunc="sum",
)

tipo_bien,EQUIPO INDUSTRIAL,EQUIPO MEDICO,MOBILIARIO
unidadservicio_clinico,,,
4 NORTE,6,0,0
ADMISION,10,0,74
ANATOMIA PATOLOGICA,11,0,0
APNEA DEL SUENO,0,0,1
BRONCOSCOPIA,0,0,1
CIRUGIA DE TORAX,0,1,1
CONGENITOS,1,0,19
CONSULTORIO EXTERNO,39,70,262
CUIDADOS PALIATIVOS,7,0,1
